### Dynamic Model

In [28]:
from dotenv import load_dotenv
import os
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from langchain.agents.structured_output import ToolStrategy

In [29]:
# Load .env variables
load_dotenv()

# Read API key
gpt_api_key = os.getenv("OPENAI_API_KEY")

# Set the environment variable that OpenAI expects
if gpt_api_key:
    print("API Loaded Succesfully!")
else:
    raise ValueError("GPT_API not found in .env file")

API Loaded Succesfully!


In [30]:
from dataclasses import dataclass
from pydantic import BaseModel, Field

# Use Pydantic instead of dataclass for better compatibility
class ResponseFormat(BaseModel):
    """Response schema for the agent."""
    llm_model_name_used: str = Field(description="The name of the LLM model used")
    answer_to_user_query: str = Field(description="The answer to the user's question")


In [31]:

basic_model = ChatOpenAI(model="gpt-4o-mini")
advanced_model = ChatOpenAI(model="gpt-4o")

@wrap_model_call
def dynamic_model_selection(request: ModelRequest, handler) -> ModelResponse:
    """Choose model based on conversation complexity."""
    message_count = len(request.state["messages"])

    if message_count > 10:
        # Use an advanced model for longer conversations
        model = advanced_model
    else:
        model = basic_model

    ##below is my made code to store the LLM model name so we can see it in output
    request.state["selected_model_name"] = getattr(model, "model", getattr(model, "name", None)) or str(model)

    return handler(request.override(model=model))



#### What is message_count?

##### This counts how many back-and-forth messages have been exchanged in the conversation, not words.

##### Conversation 1: Single message
messages = [
    {"role": "user", "content": "What is the color of the sun? One word answer only"}
]
##### message_count = 1 (only 1 message)

------------------------------------------------------------------------------------------------------

##### Conversation 2: Multiple messages

messages = [

    {"role": "user", "content": "Hi"},

    {"role": "assistant", "content": "Hello! How can I help?"},

    {"role": "user", "content": "Tell me about Python"},

    {"role": "assistant", "content": "Python is a programming language..."},

    {"role": "user", "content": "What about machine learning?"},

]

##### message_count = 5 (5 messages total)



#### The idea:

1. Short conversations (≤10 messages) → Use cheaper, faster model (gpt-4o-mini)
2. Long conversations (>10 messages) → Use smarter, more expensive model (gpt-4o) because complex discussions might need better reasoning

In [ ]:
agent = create_agent(
    model=basic_model,  # Default model
    middleware=[dynamic_model_selection],
    system_prompt = "Answer with only one word. No more than one word"
)

In [33]:
response = agent.invoke(
    {"messages": [{"role": "user", "content": "What is the color is sun. One word answer only"}]},
    response_format=ToolStrategy(ResponseFormat)
)

In [34]:
# Extract the information
final_message = response['messages'][-1]
model_used = final_message.response_metadata.get('model_name', 'Unknown')
answer = final_message.content

print(f"\n{'='*50}")
print(f"Model Used: {model_used}")
print(f"Answer: {answer}")
print(f"{'='*50}")


Model Used: gpt-4o-mini-2024-07-18
Answer: White.


##### Invoking the bigger model with 10+ messages

In [38]:
conversation_messages = [
    {"role": "user", "content": "What is the color of Sun"},
    {"role": "user", "content": "What is the color of Space"},
    {"role": "user", "content": "What's the weather like?"},
    {"role": "user", "content": "What is Machine"},
    {"role": "user", "content": "Tell me about Python."},
    {"role": "user", "content": "What is a programming language."},
    {"role": "user", "content": "What about JavaScript?"},
    {"role": "user", "content": "What is Marvel"},
    {"role": "user", "content": "How about databases?"},
    {"role": "user", "content": "What store data."},
    {"role": "user", "content": "What is the color of the Moon?"}  # 11th message
    
]

In [39]:
response = agent.invoke({"messages": conversation_messages}, response_format=ToolStrategy(ResponseFormat))

In [46]:
# Extract the information
final_message = response['messages'][-1]
model_used = final_message.response_metadata.get('model_name', 'Unknown')
answer = final_message.content

print(f"\n{'='*50}")
print(f"Model Used: {model_used}")
print(f"Answer: {answer}")
print(f"{'='*50}")


Model Used: gpt-4o-2024-08-06
Answer: The color of the Moon as seen from Earth can vary based on several factors, including its position in the sky, atmospheric conditions, and the time of observation. Generally, the Moon appears:

1. **Gray or White**: When high in the sky, the Moon usually appears to be a grayish-white color due to the reflection of sunlight off its surface, which is composed of various minerals.

2. **Yellow or Orange**: When the Moon is low on the horizon, such as during a moonrise or moonset, it can appear yellow, orange, or even reddish. This is due to Rayleigh scattering, the same phenomenon that causes sunsets to be red or orange. The Earth's atmosphere scatters shorter blue wavelengths of light more than the longer red wavelengths, causing the Moon to take on a warmer hue.

3. **Reddish**: During a total lunar eclipse, the Moon can appear reddish or copper-colored, often referred to as a "blood moon." This is due to sunlight being filtered and refracted by th